In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('./data/MNIST_data/', one_hot=True)

Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(init_random_dist)

In [5]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

In [6]:
def conv2d(x,W):
    # x --> [batch, H, W, Channels]
    # W --> [filter H, filter W, Channels IN, Channels OUT]
    return tf.nn.conv2d(x, W, strides = [1,1,1,1], padding='SAME')

In [7]:
def max_pool_2by2(x):
    #x --> [batch, h, w, c]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [8]:
#conv layer
def convolutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W)+b)

In [9]:
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b    

In [10]:
#placeholders
x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [11]:
#layers
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [26]:
convo_1 = convolutional_layer(x_image, shape=[7,7,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [27]:
# convo_2 = convolutional_layer(convo_1_pooling, shape=[5,5,32,64])
# convo_2_pooling = max_pool_2by2(convo_2)

In [36]:
convo_2_flat = tf.reshape(convo_1_pooling, [-1,14*14*32])

In [37]:
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

In [38]:
#dropout
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

In [39]:
y_pred = normal_full_layer(full_one_dropout, 10)

In [40]:
#LOSS FUNCTION
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [41]:
#OPTMIZER
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [42]:
init = tf.global_variables_initializer()

In [43]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(train, feed_dict = {x:batch_x, y_true:batch_y, hold_prob:0.5})
        
        if i%100 == 0:
            print('ON STEP: {}'. format(i))
            print('ACCURACY: ')
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            print(sess.run(acc, feed_dict={x:mnist.test.images, y_true:mnist.test.labels, hold_prob:1.0}))

ON STEP: 0
ACCURACY: 
0.2061
ON STEP: 100
ACCURACY: 
0.9032
ON STEP: 200
ACCURACY: 
0.9315
ON STEP: 300
ACCURACY: 
0.9576
ON STEP: 400
ACCURACY: 
0.9605
ON STEP: 500
ACCURACY: 
0.9699
ON STEP: 600
ACCURACY: 
0.9679
ON STEP: 700
ACCURACY: 
0.9751
ON STEP: 800
ACCURACY: 
0.9767
ON STEP: 900
ACCURACY: 
0.98
ON STEP: 1000
ACCURACY: 
0.9787
ON STEP: 1100
ACCURACY: 
0.9824
ON STEP: 1200
ACCURACY: 
0.9789
ON STEP: 1300
ACCURACY: 
0.9771
ON STEP: 1400
ACCURACY: 
0.9824
ON STEP: 1500
ACCURACY: 
0.9852
ON STEP: 1600
ACCURACY: 
0.9833
ON STEP: 1700
ACCURACY: 
0.9855
ON STEP: 1800
ACCURACY: 
0.9844
ON STEP: 1900
ACCURACY: 
0.9859
ON STEP: 2000
ACCURACY: 
0.985
ON STEP: 2100
ACCURACY: 
0.9866
ON STEP: 2200
ACCURACY: 
0.9876
ON STEP: 2300
ACCURACY: 
0.9873
ON STEP: 2400
ACCURACY: 
0.9871
ON STEP: 2500
ACCURACY: 
0.9882
ON STEP: 2600
ACCURACY: 
0.986
ON STEP: 2700
ACCURACY: 
0.985
ON STEP: 2800
ACCURACY: 
0.9866
ON STEP: 2900
ACCURACY: 
0.9893
ON STEP: 3000
ACCURACY: 
0.9886
ON STEP: 3100
ACCURACY: 


In [ ]:
print(batch_x)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
batch_x.shape

In [ ]:
mnist.test.images.shape